In [1]:
from specklepy.api.client import SpeckleClient
from specklepy.api.credentials import get_default_account, get_local_accounts

In [11]:
# initialise the client
# client = SpeckleClient(host="your-server.com") # or whatever your host is
client = SpeckleClient(host="speckle.xyz", use_ssl=True)

# authenticate the client with a token
account = get_default_account()
client.authenticate(token=account.token)

In [12]:
# create a new stream. this returns the stream id
new_stream_id = client.stream.create(name="a shiny new stream")

In [13]:
# use that stream id to get the stream from the server
new_stream = client.stream.get(id=new_stream_id)

In [15]:
from specklepy.objects import Base
from specklepy.objects.geometry import Point

class Block(Base):
    length: float = 1.0
    width: float = 1.0
    height: float = 1.0
    origin: Point = Point()

    def __init__(self, **kwargs) -> None:
        super().__init__(**kwargs)
        # mark the origin as a detachable attribute
        self.add_detachable_attrs("origin")

In [38]:
from specklepy.transports.server import ServerTransport
from specklepy.transports.memory import MemoryTransport
from specklepy.transports.sqlite import SQLiteTransport
from specklepy.api import operations

# here's the data you want to send
block = Block(length=2, height=5)

# next create a server transport - this is the vehicle through which you will send and receive
transport = ServerTransport(client=client, stream_id=new_stream_id)

# this serialises the block and sends it to the transport
hash = operations.send(base=block, transports=[transport])

# you can now create a commit on your stream with this object
commid_id = client.commit.create(
    new_stream_id, 
    hash, 
    "this is a block I made in speckle-py",
    )

In [39]:
# this receives the object back from the transport.
# the received data will be deserialised back into a `Block` 
received_base = operations.receive(obj_id=hash, remote_transport=transport)
received_base.height

5.0

In [44]:
transport.get_object

<bound method ServerTransport.get_object of ServerTransport(url='https://speckle.xyz', stream_id='db01c43fbd', saved_obj_count=0, session=<requests.sessions.Session object at 0x7fed68f2f790>, _batch_sender=<specklepy.transports.server.batch_sender.BatchSender object at 0x7fed68f2f6d0>)>

In [25]:
# get list of commits
client.commit.list(new_stream_id)

[]